## Tutorial on how to use Parcels on 3D C grids

One of the features of Parcels is that it can directly and natively work with `Field` data discretised on C-grids. These C grids are very popular in OGCMs, so velocity fields outputted by OGCMs are often provided on such grids, except if they have been firstly re-interpolated on a A grid.

More information about C-grid interpolation can be found in [Delandmeter et al., 2019](https://www.geosci-model-dev-discuss.net/gmd-2018-339/).
An example of such a discretisation is the NEMO model, which is one of the models supported in Parcels. A tutorial teaching how to [interpolate 2D data on a NEMO grid](https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_nemo_curvilinear.ipynb) is available within Parcels.

Here, we focus on 3D fields. Basically, it is a straightforward extension of the 2D example, but it is very easy to do a mistake in the setup of the vertical discretisation that would affect the interpolation scheme.

### Preliminary comments

*How to know if your data is discretised on a C grid?* The best way is to read the documentation that comes with the data. Alternatively, an easy check is to assess the coordinates of the U, V and W fields: for an A grid, U, V and W are distributed on the same nodes, such that the coordinates are the same. For a C grid, there is a shift of half a cell between the different variables.

*What about grid indexing?* Since the C-grid variables are not located on the same nodes, there is not one obvious way to define the indexing, i.e. where is `u[k,j,i]` compared to `v[k,j,i]` and `w[k,j,i]`. In Parcels, we use the same notation as in NEMO: see [horizontal indexing](https://www.nemo-ocean.eu/doc/img360.png) and [vertical indexing](https://www.nemo-ocean.eu/doc/img362.png).
It is important that you check if your data is following the same notation. Otherwise, you should re-index your data properly (this can be done within Parcels, there is no need to regenerate new netcdf files).

### How to create a 3D NEMO `dimensions` dictionary?

In the following, we will show how to create the `dimensions` dictionary for 3D NEMO simulations. What you require is a 'mesh_mask' file, which in our case is called `coordinates.nc` but in some other versions of NEMO has a different name. In any case, it will have to contain the variables `glamf`, `gphif` and `depthw`, which are ***EXPLAIN WHAT VARIABLES ARE*** respectively. Note that `depthw` is not part of the mesh_mask file, but is in the same file as the w data (`wfiles`).

For the C-grid interpolation in Parcels to work properly, it is important that `U`, `V` and `W` are on the same grid.

The code below is an example of how to create a3D simulation with particles, starting in the mouth of the river Rhine, and advecting them through the North Sea using the `AdvectionRK4_3D`

In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D
from glob import glob
import numpy as np
from datetime import timedelta as delta
from os import path

data_path = 'NemoNorthSeaORCA025-N006_data/'
ufiles = sorted(glob(data_path+'ORCA025-N06_200001??d05U.nc'))
vfiles = sorted(glob(data_path+'ORCA025-N06_200001??d05V.nc'))
wfiles = sorted(glob(data_path+'ORCA025-N06_200001??d05W.nc'))
mesh_mask = data_path + 'coordinates.nc'

filenames = {'U': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': ufiles},
             'V': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': vfiles},
             'W': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': wfiles}}

variables = {'U': 'uo',
             'V': 'vo',
             'W': 'wo'}
dimensions = {'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}}

fieldset = FieldSet.from_nemo(filenames, variables, dimensions)
fieldset.U.vmax = 5
fieldset.V.vmax = 5
fieldset.W.vmax = 5

# meshgrid containing 5x5 points uniformly distributed in a [0,1]x[0,1] quad
vec = np.linspace(0, 1, 5)
xsi, eta = np.meshgrid(vec, vec)

# get particles in Rhine estuary
lonCorners = [2.96824026, 3.22713804, 3.26175451, 3.002671]
latCorners = [51.60693741, 51.58454132, 51.73711395, 51.759758]
lon_r = (1-xsi)*(1-eta) * lonCorners[0] + xsi*(1-eta) * lonCorners[1] + \
        xsi*eta * lonCorners[2] + (1-xsi)*eta * lonCorners[3]
lat_r = (1-xsi)*(1-eta) * latCorners[0] + xsi*(1-eta) * latCorners[1] + \
        xsi*eta * latCorners[2] + (1-xsi)*eta * latCorners[3]

lonp = lon_r.flatten()
latp = lat_r.flatten()
depthp = np.ones(latp.shape)

pset = ParticleSet.from_list(fieldset, JITParticle, lon=lonp, lat=latp, depth=depthp)
kernels = pset.Kernel(AdvectionRK4_3D)
pset.execute(kernels, runtime=delta(days=4), dt=delta(hours=6))

for p in pset:
    print(p)


         It will be opened with no decoding. Filling values might be wrongly parsed.
INFO: Compiled JITParticleAdvectionRK4_3D ==> /var/folders/h0/01fvrmn11qb62yjw7v1kn62r0000gq/T/parcels-503/0e69c0fcf2d0206b75ee09df2079b491.so


P[0](lon=3.330627, lat=51.771065, depth=1.013004, time=345600.000000)
P[1](lon=3.429670, lat=51.770466, depth=0.942356, time=345600.000000)
P[2](lon=3.516687, lat=51.766171, depth=0.902212, time=345600.000000)
P[3](lon=3.587906, lat=51.770084, depth=0.824910, time=345600.000000)
P[4](lon=3.638291, lat=51.767937, depth=0.761362, time=345600.000000)
P[5](lon=3.326173, lat=51.803127, depth=1.002215, time=345600.000000)
P[6](lon=3.420991, lat=51.801682, depth=0.919173, time=345600.000000)
P[7](lon=3.510124, lat=51.798275, depth=0.876406, time=345600.000000)
P[8](lon=3.585248, lat=51.807373, depth=0.806789, time=345600.000000)
P[9](lon=3.641669, lat=51.818974, depth=0.737283, time=345600.000000)
P[10](lon=3.332299, lat=51.836002, depth=0.991055, time=345600.000000)
P[11](lon=3.423283, lat=51.833839, depth=0.907191, time=345600.000000)
P[12](lon=3.503452, lat=51.829502, depth=0.849505, time=345600.000000)
P[13](lon=3.580498, lat=51.837696, depth=0.792666, time=345600.000000)
P[14](lon=3.6452